# CppClass

Another thing we want to address is the code inside our `Greeter`.

We are currently writing a lot of boilerplate there.

We have the `__enter__` method, the unused `__exit__` arguments, and pushing
the instance into the dtor stack.

In [ ]:
class Greeter:
    def __init__(self, name):
        push_dtor(self)

        self.name = name
        print(f"Hello, {self.name}!")

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        print(f"Goodbye, {self.name}.")

In [ ]:
from cpp import magic


class CppClass:
    def __init__(self, *args, **kwargs):
        push_dtor(self)

        ctor = getattr(self, self.__class__.__name__, None)
        if ctor:
            ctor(*args, **kwargs)

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        dtor = getattr(self, "_" + self.__class__.__name__, None)
        if dtor:
            dtor()


class Greeter(CppClass):
    def Greeter(self, name):
        self.name = name
        print(f"Hello, {self.name}!")

    def _Greeter(self):
        print(f"Goodbye, {self.name}.")


def main():
    greeter1 = Greeter(1)
    greeter2 = Greeter(2)


That's great.
Now as we add more classes, we don't need to handle all that annoying
dtor-stack stuff.

But... We're still missing something.
We decorated all free functions with `cpp_function`, but we still need to
decorate all of our member functions.

In [ ]:
def decorate_object_methods(obj):
    for name, value in inspect.getmembers(obj):
        if name.startswith("__"):
            continue

        if not inspect.isroutine(value):
            continue

        setattr(self, name, cpp_function(value))


class CppClass:
    def __init__(self, *args, **kwargs):
        push_dtor(self)

        decorate_object_methods(self)

        ctor = getattr(self, self.__class__.__name__, None)
        if ctor:
            ctor(*args, **kwargs)

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        dtor = getattr(self, "_" + self.__class__.__name__, None)
        if dtor:
            dtor()


This is similar to what we did with the modules, but this
time we check for special methods, as we don't wanna decorate them.

Last but not list - we want to make it truly implicit.

Currently, our `Greeter` class subclasses `CppClass` to get the
relevant functionality.

In essence, we're injecting 3 methods into our `Greeter` class
using inheritance.
But we can also do this using a decorator.
We take the class, make the relevant modifications,
then return the modified version.

In [ ]:
Greeter.__init__ = __init__
Greeter.__enter__ = __enter__
Greeter.__exit__ = __exit__


In [ ]:

def cpp_class(cls):

    def __init__(self, *args, **kwargs):
        push_dtor(self)

        decorate_object_methods(self)

        ctor = getattr(self, self.__class__.__name__, None)
        if ctor:
            ctor(*args, **kwargs)

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        dtor = getattr(self, "_" + self.__class__.__name__, None)
        if dtor:
            dtor()

    cls.__init__ = __init__
    cls.__enter__ = __enter__
    cls.__exit__ = __exit__

    return cls


@cpp_class
class Greeter:
    ...

As a next step, we move the decoration out of the constructor,
and perform it on the class object instead of the instance.

Additionally, as we can no longer tell our object is a `CppClass`
using inheritance checks, we add a flag to indicate it.

In [ ]:
def cpp_class(cls):
    decorate_object_methods(cls)

    for name in getattr(cls, '__annotations__', {}):
        member = CppMember()
        member.__set_name__(cls, name)
        setattr(cls, name, member)

    def __init__(self, *args, **kwargs):
        push_dtor(self)
        ctor = getattr(self, self.__class__.__name__, None)
        if ctor:
            ctor(*args, **kwargs)

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        dtor = getattr(self, "_" + self.__class__.__name__, None)
        if dtor:
            dtor()

    setattr(cls, '__init__', __init__)
    setattr(cls, '__enter__', __enter__)
    setattr(cls, '__exit__', __exit__)

    setattr(cls, '__cpp_class__', True)

    return cls

def is_cpp_class(obj):
    return hasattr(obj, '__cpp_class__')

And lastly, we modify our `_magic()` method to decorate classes,
as well as functions.

In [ ]:
def decorate_module_classes(module):
    for name, value in inspect.getmembers(module):
        if not inspect.isclass(value):
            continue

        # Only convert functions that were defined in the importing file.
        # We don't want to convert library imports and the likes of those.
        if inspect.getmodule(value) != module:
            continue

        setattr(module, name, cpp_class(value))


def _magic():
    calling_module = get_calling_module()
    name = calling_module.__name__
    path = calling_module.__file__
    if hasattr(calling_module, IMPORT_FLAG):
        return

    imported_module = import_by_path(name, path)

    decorate_module_functions(imported_module)
    decorate_module_classes(imported_module)

    if imported_module.__name__ == "__main__":
        sys.exit(imported_module.main())

In [ ]:
def decorate_module_members(module):
    for name, value in inspect.getmembers(module):
        if inspect.getmodule(value) != module:
            continue

        if inspect.isroutine(value):
            setattr(module, name, cpp_function(value))

        elif inspect.isclass(value):
            setattr(module, name, cpp_class(value))

And with that all of our classes are converted to C++ classes automatically!

In [ ]:
from cpp import magic

class Greeter:
    def Greeter(self, name):
        self.name = name
        print(f"Hello, {self.name}!")

    def _Greeter(self):
        print(f"Goodbye, {self.name}.")

def main():
    greeter1 = Greeter(1)
    greeter2 = Greeter(2)